In [1]:
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation
from keras.initializers import VarianceScaling
from keras.optimizers import Adam
from keras.utils import np_utils, multi_gpu_model
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.utils import class_weight
from keras.models import load_model
np.set_printoptions(suppress=True)

trainD = np.load("stevensTrainingData/trainingData.npy")
trainL = np.load("stevensTrainingData/trainingLabel.npy")
testD = np.load("stevensTrainingData/testingData.npy")
testL = np.load("stevensTrainingData/testingLabel.npy")


trainData = trainD / 255
testData = testD / 255


trainData_normalize = trainD.reshape(trainData.shape[0], trainData.shape[1], trainData.shape[2], 3).astype('uint8')
trainLabel_onehot = np_utils.to_categorical(trainL, 4)
testData_normalize = testD.reshape(testData.shape[0], testData.shape[1], testData.shape[2], 3).astype('uint8')
testLabel_onehot = np_utils.to_categorical(testL, 4)
print('Train Data:', trainData_normalize.shape)
print('Train Label: ', trainLabel_onehot.shape)
print('Test Data: ', testData_normalize.shape)
print('Test Label: ', testLabel_onehot.shape)

#建立CNN模型
model = Sequential()
model.add(Conv2D(filters = 16, kernel_size = (2, 2), input_shape = (trainData_normalize.shape[1], trainData_normalize.shape[2], 3), data_format = "channels_last"))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2), data_format = "channels_last"))

model.add(Conv2D(filters = 32, kernel_size = (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (2, 2), data_format = "channels_last"))

model.add(Conv2D(filters = 64, kernel_size = (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (3, 3), data_format = "channels_last"))

model.add(Conv2D(filters = 64, kernel_size = (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size = (3, 3), data_format = "channels_last"))

#建立分類模型
model.add(Flatten())
model.add(Dense(128, activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(32, activation = 'relu'))
model.add(Dense(16, activation = 'relu'))
model.add(Dense(4, activation = "softmax"))

print(model.summary())


adam = Adam(lr = 0.001)
model.compile(optimizer = adam, loss = "categorical_crossentropy", metrics=['accuracy'])

early_stop = EarlyStopping(patience=20)

train_history_1 = model.fit(x = trainData_normalize,
                            y = trainLabel_onehot,
                            epochs=20,
                            validation_data=(testData_normalize, testLabel_onehot),
                            batch_size=100, 
                            verbose=1
                           )

evaluation = model.evaluate(x = testData_normalize, y = testLabel_onehot)
print('Loss: {:.3f}, Accuracy: {:.3f}'.format(evaluation[0], evaluation[1]))


model.save("human_detection_model.h5")

print('Finish training!')

import pylab as plt
def history_display(hist, train, validation):
    plt.plot(hist.history[train])
    plt.plot(hist.history[validation])
    plt.title('Train History')
    plt.ylabel(train)
    plt.xlabel('Epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show
    
def show_plot(flag, hist):
    if flag == 'acc':
        history_display(hist, 'acc', 'val_acc')
    elif flag == 'loss':
        history_display(hist, 'loss', 'val_loss')
    else:
        print('Invalid!')

Using TensorFlow backend.


Train Data: (280, 192, 256, 3)
Train Label:  (280, 4)
Test Data:  (120, 192, 256, 3)
Test Label:  (120, 4)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 191, 255, 16)      208       
_________________________________________________________________
activation_1 (Activation)    (None, 191, 255, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 95, 127, 16)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 93, 125, 32)       4640      
_________________________________________________________________
activation_2 (Activation)    (None, 93, 125, 32)       0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 46, 62, 32)        0         
___________________________________

In [13]:
import numpy as np
import pandas as pd
import os
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv1D, MaxPooling1D, AveragePooling1D, Dropout, Conv2D, MaxPooling2D
from keras.layers import BatchNormalization, Activation
from keras.initializers import VarianceScaling
from keras.optimizers import Adam
from keras.utils import np_utils, multi_gpu_model
import tensorflow as tf
from keras.callbacks import TensorBoard, EarlyStopping
from sklearn.utils import class_weight
from keras.models import load_model

model = load_model('human_detection_model_0875.h5')
imagenpy = np.load("sampleImage.npy")
imagenpy = imagenpy.reshape((1, imagenpy.shape[0], imagenpy.shape[1], imagenpy.shape[2]))
result = model.predict(imagenpy)



In [14]:
print(result)

[[0.12593701 0.04073597 0.41777843 0.41554862]]
